In [179]:
from pysam import VariantFile
from sklearn.cluster import KMeans

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [196]:
vcf_file = 'DRR131565.variants.HC_init.wAnnot.vcf'
mca_file = 'MVP_MCA_results.csv'

Parse VCF File and get variants

In [197]:
def getGenesVariantList(vcf_file):
    
    vcf = VariantFile(vcf_file)
    
    variants = set()
        
    for rec in vcf.fetch():
         for gene_name in rec.info["GENE"]:
                variants.add(gene_name)

    return variants

Apply clustering to MCA to identify genes closest to the closest experiment

In [189]:
# Uses a method to dynamically determine number of clusters, 
# and then runs kmeans
def getKMeanLabels(X):
    
    ##################
    # Code borrowed from Jessica Temporal 
    # https://jtemporal.com/kmeans-and-elbow-method/
    def calculate_wcss(data):
            wcss = []
            for n in range(2, 21):
                kmeans = KMeans(n_clusters=n)
                kmeans.fit(X=data)
                wcss.append(kmeans.inertia_)

            return wcss

    def optimal_number_of_clusters(wcss):
        x1, y1 = 2, wcss[0]
        x2, y2 = 20, wcss[len(wcss)-1]

        distances = []
        for i in range(len(wcss)):
            x0 = i+2
            y0 = wcss[i]
            numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
            denominator = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
            distances.append(numerator/denominator)

        return distances.index(max(distances)) + 2
    
    # calculating the within clusters sum-of-squares for 19 cluster amounts
    sum_of_squares = calculate_wcss(X)

    # calculating the optimal number of clusters
    n = optimal_number_of_clusters(sum_of_squares)
    ###################
    
    kmeans = KMeans(n_clusters=n)
    kmeans.fit(X)
    y_kmeans = kmeans.predict(X)
    
    return y_kmeans, kmeans.cluster_centers_

In [206]:
# get 2D distance between A and B
def getDist(A,B):
    return math.sqrt((A[0]-B[0])**2 + (A[1]-B[1])**2)

# gets the minimum distance from point to 
# a center in centers
def getMinDist(point, centers):
    
    min_dist = getDist(point,centers[0])
    min_idx = 0
    
    for idx, center in enumerate(centers):
        dist = getDist(point, center)
        if dist < min_dist: 
            min_dist = dist
            min_idx = idx
            
    return (min_idx, min_dist)

In [218]:
def getGenesOfInterestFromMCA(mca_file):
    
    mca_df = pd.read_csv('MVP_MCA_results.csv')

    gene_df = mca_df[mca_df['AssociationType'].isin(['Gene' , 'DrugTerm'])]
    no_gene_df = mca_df[~mca_df['AssociationType'].isin(['Gene', 'DrugTerm'])]
    
    X = np.array(list([list(a) for a in zip(gene_df['x'], gene_df['y'])]))
    
    # cluster genes
    y_kmeans, cluster_centers = getKMeanLabels(X)
    gene_df['cluster_center'] = y_kmeans
    
    # Display clustering
    # plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
    # plt.show()
    
    # add the distance from the closest center to each 
    no_gene_df['closestClusterCenter'] = \
        no_gene_df.apply(lambda row: getMinDist([row.x, row.y], cluster_centers)[0], axis=1)
    
    no_gene_df['distToClusterCenter'] = \
        no_gene_df.apply(lambda row: getMinDist([row.x, row.y], cluster_centers)[1], axis=1)
    
    no_gene_df = no_gene_df.sort_values(by=['distToClusterCenter'])
    
    cluster_of_interest = no_gene_df['closestClusterCenter'].iloc[0]
    
    return list(gene_df[gene_df['cluster_center']==cluster_of_interest]['Value'].unique())

<ipython-input-218-45beca6bd270>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['cluster_center'] = y_kmeans
<ipython-input-218-45beca6bd270>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_gene_df['closestClusterCenter'] = \
<ipython-input-218-45beca6bd270>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [224]:
def getGenesOfInterest(gene_list, variants):
    
    # currently just taking the intersection, but
    # can be made more sophisticated
    return set(gene_list).intersection(variants)

In [225]:
variants = getGenesVariantList(filename)
gene_list = getGenesOfInterestFromMCA(mca_file)

<ipython-input-218-45beca6bd270>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['cluster_center'] = y_kmeans
<ipython-input-218-45beca6bd270>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_gene_df['closestClusterCenter'] = \
<ipython-input-218-45beca6bd270>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [226]:
getGenesOfInterest(gene_list, variants)

{'ATL2',
 'EFCAB11',
 'FBXO34',
 'KIF22',
 'KLHL29',
 'MRPS5',
 'PAQR5',
 'RGS20',
 'SACM1L',
 'ZFAND5'}